In [ ]:
# !git clone https://github.com/ultralytics/ultralytics
# !pip install ultralytics
# cd ultralytics
# !pip install -r requirements.txt
# cd ..

In [ ]:
# # YOLOv8n 모델 다시 다운로드
# !wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt

In [1]:
import torch

print(torch.cuda.memory_summary())
torch.cuda.empty_cache()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [2]:
import torch

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 가능 여부: {torch.cuda.is_available()}")
print(f"현재 사용 중인 CUDA 버전: {torch.version.cuda}")
print(f"CUDNN 버전: {torch.backends.cudnn.version()}")

PyTorch 버전: 2.5.1+cu124
CUDA 가능 여부: True
현재 사용 중인 CUDA 버전: 12.4
CUDNN 버전: 90100


In [3]:
!nvidia-smi

Tue Nov 12 06:54:10 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:C1:00.0 Off |                  Off |
|  0%   45C    P8             24W /  450W |       4MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import torch
print(torch.cuda.is_available())  # True가 출력되어야 GPU 사용 가능

True


- 라벨링 데이터가 골프공 하나일때만 돌리기!!

In [6]:
import os
dir = ['train', 'valid', 'test']

def change_class_number(label_dir, new_class_number):
    # 디렉토리 내 모든 .txt 파일을 순회
    label_dir = f'./merged_golfball/{label_dir}/labels'
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            label_path = os.path.join(label_dir, label_file)

            # 라벨 파일 읽기
            with open(label_path, 'r') as file:
                lines = file.readlines()

            # 각 줄의 클래스 번호를 new_class_number로 변경
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) > 0:
                    parts[0] = str(new_class_number)  # 클래스 번호를 new_class_number로 변경
                    new_lines.append(' '.join(parts))

            # 수정된 내용을 다시 라벨 파일에 쓰기
            with open(label_path, 'w') as file:
                file.write('\n'.join(new_lines) + '\n')

    print(f"라벨링 번호가 모두 {new_class_number}로 변경되었습니다.")

for label_dir in dir:
    change_class_number(label_dir, 0)

라벨링 번호가 모두 0로 변경되었습니다.
라벨링 번호가 모두 0로 변경되었습니다.
라벨링 번호가 모두 0로 변경되었습니다.


# 추가 학습 실험 하시려면 여기만 건들이세요!!!

In [ ]:
from ultralytics import YOLO

# YOLOv8n 모델 로드 (사전학습된 모델)
model = YOLO('yolov8n.pt')

# 모델 학습 시작
model.train(
    data='./merged_golfball_with_golf_club/data.yaml',  # YAML 파일 경로
    epochs=200, 
    imgsz=640, # 이미지 크기
    batch=16,
    name='yolov8n_golf_ball_merged_golfball_with_golf_club_150epoch_16batch_10patience',
    patience=10,  # 10 epoch 동안 개선이 없으면 조기 종료
    augment=True  # 데이터 증강 활성화, 여러 증강 기법들이 학습 과정에서 자동으로 적용, 여기에는 회전, 크기 조정, 좌우 반전, 밝기 및 대비 조정 등의 다양한 변환이 포함
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
New https://pypi.org/project/ultralytics/8.3.29 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.27 🚀 Python-3.9.20 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24111MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./merged_golfball/data.yaml, epochs=100, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_golf_ball_merged_data_100epoch_8batch_10patience, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=Fals

100%|██████████| 755k/755k [00:00<00:00, 26.5MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 18                  -1  1    123648  ultralytics.nn.modules.block.C2f             [192, 128, 1]                 
 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 225 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/yolov8n_golf_ball_merged_data_100epoch_8batch_10patience', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 15.5MB/s]


...

                   all        448        715      0.964      0.903      0.959      0.786


EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 77, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

87 epochs completed in 0.753 hours.
Optimizer stripped from runs/detect/yolov8n_golf_ball_merged_data_100epoch_8batch_10patience/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/yolov8n_golf_ball_merged_data_100epoch_8batch_10patience/weights/best.pt, 6.3MB

Validating runs/detect/yolov8n_golf_ball_merged_data_100epoch_8batch_10patience/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.9.20 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24111MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:02<00:00, 11.21it/s]


                   all        448        715       0.93      0.922      0.959      0.782
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/yolov8n_golf_ball_merged_data_100epoch_8batch_10patience


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f0d541fa2b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [8]:
# 검증 데이터셋으로 모델 평가
results = model.val()

Ultralytics 8.3.27 🚀 Python-3.9.20 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24111MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /root/YOLOv8n_Golfball/merged_golfball/valid/labels.cache... 448 images, 1 backgrounds, 0 corrupt: 100%|██████████| 448/448 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:03<00:00, 18.29it/s]


                   all        448        715      0.931      0.922      0.959      0.783
Speed: 0.3ms preprocess, 3.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/yolov8n_golf_ball_merged_data_100epoch_8batch_10patience2


In [7]:
import pandas as pd
results_df = pd.read_csv('/root/YOLO_GOLF/runs/detect/yolov8n_golf_ball_100epoch_10patience_8batch/results.csv')
results_df

,epoch,time,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),val/box_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,17.7974,1.01952,1.53185,1.20647,0.80259,0.77491,0.79671,0.49691,1.16135,1.41043,1.37551,0.000664,0.000664,0.000664
1,2,31.7419,1.07298,1.17419,1.23323,0.85813,0.76864,0.82944,0.57584,1.04713,1.29217,1.25514,0.001318,0.001318,0.001318
2,3,45.4636,1.06847,1.02328,1.23297,0.84013,0.79834,0.83440,0.61019,0.95966,0.88772,1.19697,0.001958,0.001958,0.001958
3,4,59.0101,1.04322,0.94504,1.22133,0.89720,0.82873,0.84483,0.62366,0.98856,0.80684,1.18422,0.001941,0.001941,0.001941
4,5,72.5478,1.01203,0.88575,1.19647,0.87866,0.82597,0.86330,0.60810,1.00340,0.73188,1.20598,0.001921,0.001921,0.001921
5,6,85.9300,0.99668,0.85178,1.19100,0.88926,0.84293,0.88276,0.66772,0.93437,0.65269,1.16236,0.001901,0.001901,0.001901
6,7,99.3824,0.98235,0.77483,1.18446,0.88562,0.81281,0.86249,0.63525,0.94674,0.78779,1.15482,0.001881,0.001881,0.001881
7,8,112.5970,0.95683,0.77778,1.16248,0.89974,0.81806,0.88123,0.66109,0.87685,0.70493,1.15583,0.001861,0.001861,0.001861
8,9,126.0160,0.92254,0.74623,1.14646,0.94185,0.85635,0.90279,0.68057,0.90936,0.59504,1.13064,0.001842,0.001842,0.001842
9,10,139.4550,0.92967,0.73042,1.14381,0.89874,0.85810,0.89501,0.68420,0.86295,0.59679,1.10322,0.001822,0.001822,0.001822


In [6]:
# 이미지에 대한 예측 수행
results = model.predict('/root/YOLO_GOLF/GolfBallDetector-10/test/images/6ced8d90b264f0ac_jpg.rf.d6f2a323158e8412ae30ee43542f1a4a.jpg', conf=0.25)
results


image 1/1 /root/YOLO_GOLF/GolfBallDetector-10/test/images/6ced8d90b264f0ac_jpg.rf.d6f2a323158e8412ae30ee43542f1a4a.jpg: 640x640 2 golfballs, 82.4ms
Speed: 1.8ms preprocess, 82.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'golfball'}
 obb: None
 orig_img: array([[[201, 171, 144],
         [202, 173, 146],
         [204, 175, 148],
         ...,
         [186, 164, 129],
         [186, 164, 129],
         [184, 162, 127]],
 
        [[206, 176, 151],
         [207, 178, 151],
         [207, 180, 154],
         ...,
         [184, 162, 127],
         [186, 161, 127],
         [185, 163, 128]],
 
        [[211, 183, 159],
         [211, 186, 160],
         [212, 186, 162],
         ...,
         [183, 159, 123],
         [187, 161, 125],
         [188, 164, 128]],
 
        ...,
 
        [[ 33,  47,  45],
         [ 34,  48,  46],
         [ 37,  51,  49],
         ...,
         [ 22,  79,  58],
         [ 21,  78,  57],
         [ 21,  78,  57]],
 
        [[ 29,  43,  41],
         [ 29,  43,  41],
         [ 29,  43,  41],
         ...,
         [ 24,  